In [9]:
import os
import json

with open(os.path.join("model", "whisper", "train_packet.json")) as f:
    train_packet_loss = json.load(f)["train_loss"]
with open(os.path.join("model", "whisper", "train_flow.json")) as f:
    train_flow_loss = json.load(f)["train_loss"]

benign_traffic_list = ["benign2"]
attack_traffic_list = ["BruteForce-Web", "BruteForce-XSS", "mirai", "infiltration", "SQL_Injection", "osscan", "ssldosA10only"]

granularity_list = ["packet", "flow"]
train_loss_list = [train_packet_loss, train_flow_loss]

In [10]:
phi = 4

for benign_traffic in benign_traffic_list:
    filename = os.path.join("result", "whisper", benign_traffic)
    for idx in range(2):
        granularity = granularity_list[idx]
        train_loss = train_loss_list[idx]
        with open(f"{filename}-{granularity}.json", "r") as f:
            result = json.load(f)
            bad_count = 0
            for dict_pair in result:
                if dict_pair["loss"] > phi * train_loss:
                    bad_count += 1
            print(f"{benign_traffic}-{granularity} accuracy: {1 - bad_count / len(result)}")

for attack_traffic in attack_traffic_list:
    filename = os.path.join("result", "whisper", attack_traffic)
    for idx in range(2):
        granularity = granularity_list[idx]
        train_loss = train_loss_list[idx]
        with open(f"{filename}-{granularity}.json", "r") as f:
            result = json.load(f)
            bad_count = 0
            for dict_pair in result:
                if dict_pair["loss"] <= phi * train_loss:
                    bad_count += 1
            if len(result) > 1:
                print(f"{attack_traffic}-{granularity} accuracy: {1 - bad_count / len(result)}")
            else:
                print(f"{attack_traffic}-{granularity} accuracy: empty result")

benign2-packet accuracy: 0.5897380385364798
benign2-flow accuracy: 0.9722772277227723
BruteForce-Web-packet accuracy: 1.0
BruteForce-Web-flow accuracy: 1.0
BruteForce-XSS-packet accuracy: 1.0
BruteForce-XSS-flow accuracy: 1.0
mirai-packet accuracy: 0.3630716181823157
mirai-flow accuracy: 0.6956521739130435
infiltration-packet accuracy: empty result
infiltration-flow accuracy: 0.8333333333333334
SQL_Injection-packet accuracy: 1.0
SQL_Injection-flow accuracy: empty result
osscan-packet accuracy: 0.019061583577712593
osscan-flow accuracy: empty result
ssldosA10only-packet accuracy: 1.0
ssldosA10only-flow accuracy: empty result
